In [ ]:
import mediapipe as mp
import cv2
from dollarpy import Recognizer, Template, Point
import time
import pickle
import socket
import threading
import tkinter as tk
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


In [2]:
# Load the model
model_filename = 'pinching.pkl'
with open(model_filename, 'rb') as model_file:
    templates = pickle.load(model_file)

# Initialize recognizer with loaded templates
recognizer = Recognizer(templates)
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
def get_screen_resolution():
    # Use tkinter to get the screen resolution
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    return root.winfo_screenwidth(), root.winfo_screenheight()

In [ ]:
SERVER_ADDRESS = 'DESKTOP-HOHKG61'  # Replace with the actual server address
SERVER_PORT = 8000

# Function to handle receiving messages from the server
def receive_messages(sock):
    while True:
        try:
            message = sock.recv(1024).decode('utf-8')
            if message:
                print(f"Received from server: {message}")
            else:
                break
        except Exception as e:
            print(f"Error receiving message: {e}")
            break

# Function to start the client and connect to the server
def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        client_socket.connect((SERVER_ADDRESS, SERVER_PORT))
        print("Connected to the server.")

        # Start a thread to listen for messages from the server
        threading.Thread(target=receive_messages, args=(client_socket,), daemon=True).start()

        return client_socket

    except Exception as e:
        print(f"Failed to connect to the server: {e}")
        return None

# Function to send "pinching" message to the server
def pinching_status_callback(pinching_detected, client_socket, pinch_coordinates=None):
    if pinching_detected:
        print("Pinching")
        if pinch_coordinates:
            print(f"{pinch_coordinates[0]},{pinch_coordinates[1]}")
        if client_socket:
            try:
                message = "SENDTO:1|pinching,{x},{y},".format(x=pinch_coordinates[0], y=pinch_coordinates[1])
                client_socket.send(message.encode('utf-8'))
                print(f"Sent: {message}")
            except Exception as e:
                print(f"Error sending message: {e}")
    # else:
        # print("Not pinching")

# Function to detect pinching gestures in real-time
def getPointsRealTime(pinching_detected_callback):
    cap = cv2.VideoCapture(0)  # Use camera
    client_socket = start_client()  # Start the socket client

    mp_holistic = mp.solutions.holistic
    mp_hands = mp.solutions.hands
    drawing_utils = mp.solutions.drawing_utils

    # Get the screen resolution
    screen_width, screen_height = get_screen_resolution()

    # Set up a resizable window
    window_name = "Gesture Detection"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, screen_width, screen_height)

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip the frame horizontally to mirror the camera feed
            frame = cv2.flip(frame, 1)
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw hand landmarks
            drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Check for pinch gesture
            pinching_detected = False
            pinch_coordinates = None
            if results.right_hand_landmarks or results.left_hand_landmarks:
                try:
                    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    
                    # Calculate distance between thumb and index fingertip to detect pinch
                    distance = ((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2) ** 0.5

                    if distance < 0.02:  # Adjust threshold as needed
                        pinching_detected = True
                        # Calculate pinch coordinates
                        pinch_x = int(index_tip.x * image.shape[1])
                        pinch_y = int(index_tip.y * image.shape[0])
                        pinch_coordinates = (pinch_x, pinch_y)
                        # Draw a green dot at the pinch location
                        cv2.circle(image, (pinch_x, pinch_y), 10, (0, 255, 0), -1)  # Draw green dot

                except Exception as e:
                    print(f"Error processing hand landmarks: {e}")

            # Call the callback with the current pinching status and coordinates
            pinching_detected_callback(pinching_detected, client_socket, pinch_coordinates)

            # Display the frame
            cv2.imshow(window_name, image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# Start real-time gesture detection
getPointsRealTime(pinching_status_callback)

Connected to the server.
Pinching
91,121
Sent: SENDTO:1|pinching,91,121,
Received from server: Message received!
Pinching
76,114
Sent: SENDTO:1|pinching,76,114,
Pinching
67,116
Sent: SENDTO:1|pinching,67,116,
Pinching
59,115
Sent: SENDTO:1|pinching,59,115,
Pinching
57,114
Sent: SENDTO:1|pinching,57,114,
Pinching
58,113
Sent: SENDTO:1|pinching,58,113,
Pinching
56,112
Sent: SENDTO:1|pinching,56,112,
Pinching
58,110
Sent: SENDTO:1|pinching,58,110,
Pinching
56,110
Sent: SENDTO:1|pinching,56,110,
Pinching
56,112
Sent: SENDTO:1|pinching,56,112,
Pinching
54,108
Sent: SENDTO:1|pinching,54,108,
Pinching
55,107
Sent: SENDTO:1|pinching,55,107,
Pinching
52,112
Sent: SENDTO:1|pinching,52,112,
Pinching
50,112
Sent: SENDTO:1|pinching,50,112,
Pinching
50,110
Sent: SENDTO:1|pinching,50,110,
Pinching
49,110
Sent: SENDTO:1|pinching,49,110,
Pinching
53,110
Sent: SENDTO:1|pinching,53,110,
Received from server: This is your follow-up message after 5 seconds!
Received from server: SENDTO:1|pinching,91,121,Me